# Quadratic unconstrained binary optimization (QUBO)

## Example 1

Consider a set of N positive integers (though one doesn't need to) $U = \{n_1, n_2, n_3, \dots, n_N\}$, and imagine the goal is to construct two disjoint subsets of this, called A and B, such that $A\cup B=U$ and $A\cap B=\empty$, such that the sum of the numbers in each subset is equal or as close to each other as possible.

Formally, the sets A and B are to be constructed so that

$$f(\{n_i\}) = (S_A - S_B)^2~~\text{where}$$
$$S_A = \sum_{i\in A} n_i, \quad S_B = \sum_{i\in B}n_i$$

is minimum. This is a `cost function` that has to be minimized.

What we do is, assign to each number $n_i$ a binary variable $x_i$, such that $x_i = 1$ if $n_i\in A$ or otherwise $x_i = 0$ if $n_i \in B$.

Now we can write the partial sum as
$$
S_A = \sum_{i\in U} n_i x_i \\S_B = \sum_{i\in U}n_i (1-x_i)
$$

The cost function can now be written as 

\begin{align}
f(\{n_i\}) &= (S_A - S_B)^2 = S_A^2 + S_B^2 - 2S_A S_B\\
&= (\sum_{i\in U} n_i x_i)^2 + (\sum_{i\in U}n_i (1-x_i))^2 - 2 (\sum_{i\in U} n_i x_i)\sum_{j\in U}n_j (1-x_j)\\
&= \sum_{ij\in U}\left[
    n_i n_j x_i x_j + n_i n_j (1-x_i)(1-x_j) - 2 n_i n_j x_i (1-x_j)
    \right]\\
    &= \sum_{ij\in U} n_i n_j (x_i x_j + 1 - x_i - x_j + x_i x_j - 2x_i + 2 x_i x_j)\\
    &= 4\sum_{ij\in U} n_i n_j x_i x_j - 4\sum_{ij\in U}(n_i n_j x_i) + N^2\\
    &= 4\sum_{ij\in U} n_i n_j x_i x_j - 4S\sum_{i\in U}n_i x_i + N^2 
\end{align}

Where $S_U = \sum_{i\in U} n_i$ is the sum of all the numbers in the set U. Thus we need to minimize the `f` over the discrete space of vector **x** $ = \{x_1, x_2, \dots, x_N\}$. Note that the cost function `f` is quadratic in $x_i$

$$
f(x) = 4\sum_{ij\in U} n_i n_j x_i x_j - 4S\sum_{i\in U}n_i x_i + N^2 
$$


The cost function can be expressed in matrix form as

$$
f(x) = 4 x^{T}Qx + V^{T}x + N^2
$$

where $Q_{ij} = 4n_i n_j$ and $V_i = 4S n_i$

In [43]:
from typing import Any
import numpy as np

class qubo:

    def __init__(self, ni: np.ndarray) -> None:
        self.ni = np.asarray(ni)
        self.Q = 4 * np.outer(self.ni, self.ni)
        self.sum = self.ni.sum()
        self.V = 4 * self.sum * self.ni
        self.N = self.ni.shape[0]
    def __call__(self, x: Any) -> float:
        f = x @ (self.Q @ x) + self.V.dot(x) + self.N**2
        return f
    


In [53]:
# a = np.random.random(6) * 10 + 1
a = np.array([2, 5, 9, 7, 3, 5])
a = a.astype(int)
qfunc = qubo(ni=a)
a == qfunc.ni,a

(array([ True,  True,  True,  True,  True,  True]), array([2, 5, 9, 7, 3, 5]))

In [54]:
# 2, 5, 9, 7, 3, 5
A = np.array([9,5,2])
B = np.array([7,5,3])
print((A.sum() - B.sum())**2)

1


In [55]:
x = np.random.randint(0, 2, size=6) # .astype(bool)
print(a, x, qfunc(x))
print(qfunc.ni)
print(qfunc.Q.round(3))
print(qfunc.V)

[2 5 9 7 3 5] [1 0 1 1 1 1] 5964
[2 5 9 7 3 5]
[[ 16  40  72  56  24  40]
 [ 40 100 180 140  60 100]
 [ 72 180 324 252 108 180]
 [ 56 140 252 196  84 140]
 [ 24  60 108  84  36  60]
 [ 40 100 180 140  60 100]]
[ 248  620 1116  868  372  620]
